# knowledge generation
논리지 생성 단계

# 1. 필요 라이브러리 설치
- Newspaper3k와 Selenium을 사용하여 웹 스크래핑을 수행합니다.
- Chromium-browser와 ChromeDriver는 Selenium이 웹 컨텐츠를 효과적으로 스크랩할 수 있도록 합니다.

In [27]:
!pip install newspaper3k selenium
!pip install chromedriver-autoinstaller
!sudo apt-get update
!sudo apt-get install chromium-browser


Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease          
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease       
Hit:3 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


# 2. Python 스크립트
## 1) 라이브러리 불러오기
- 웹 스크래핑을 위한 필수 라이브러리를 불러옵니다.
- newspaper는 복잡한 웹페이지에서 콘텐츠를 추출하고, BeautifulSoup와 requests는 HTML 페이지를 파싱합니다.

In [28]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


## 2) 뉴스 기사 URL 검색
네이버 뉴스를 통해 주어진 검색어로 최신 뉴스 기사 URL을 검색하고 추출합니다.

In [29]:
def get_news_urls(query):
    search_url = f"https://search.naver.com/search.naver?where=news&query={query}&sort=1"
    response = requests.get(search_url, headers={'User-agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        news_urls = [element['href'] for element in soup.find_all('a', class_='news_tit')]
        return news_urls
    else:
        print("Failed to fetch news URLs")
        return []


## 3)뉴스 기사 스크래핑
추출한 URL에서 뉴스 기사의 세부 정보를 추출합니다.

In [30]:
def scrape_news_articles(news_urls):
    articles_data = []
    for url in news_urls:
        try:
            article = Article(url)
            article.download()
            article.parse()
            articles_data.append({
                'url': url,
                'title': article.title,
                'authors': article.authors,
                'publish_date': article.publish_date,
                'text': article.text
            })
        except Exception as e:
            print(f"Failed to scrape article from {url}: {e}")
    return articles_data


## 4) 데이터 정리 및 Selenium 사용
- 서울 지하철 파업/연착/지연 검색
- DataFrame을 생성하고, 텍스트가 없거나 충분하지 않은 경우 Selenium으로 추가 데이터를 수집합니다.
- null 값이거나 텍스트 길이가 100자 이하 일때, 크롤링이 제대로 되지 않은 걸로 간주, selenium을 통해 크롤링

In [31]:
if __name__ == "__main__":
    queries = ["서울 지하철 파업", "서울 지하철 연착", "서울 지하철 지연"]
    all_articles_data = []
    for query in queries:
        news_urls = get_news_urls(query)
        articles_data = scrape_news_articles(news_urls)
        for article_data in articles_data:
            article_data['category'] = query.split(" ")[-1]
        all_articles_data.extend(articles_data)
    df = pd.DataFrame(all_articles_data)
    df = df.replace('', np.nan)

    # Selenium 설정 및 데이터 추가 수집
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')  # This can help avoid some common errors
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    for index, row in df.iterrows():
        if pd.isna(row['text']) or len(row['text']) < 100:
            driver.get(row['url'])
            try:
                fetched_text = driver.find_element(By.TAG_NAME, 'article').text
                df.at[index, 'text'] = fetched_text
            except Exception as e:
                print(f"Error fetching article from {row['url']}: {e}")
    driver.quit()


# 3. 데이터 확인

In [32]:
df.shape

(30, 6)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           30 non-null     object
 1   title         30 non-null     object
 2   authors       30 non-null     object
 3   publish_date  25 non-null     object
 4   text          30 non-null     object
 5   category      30 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB


In [34]:
df.head(30)

url  \
0   http://www.m-i.kr/news/articleView.html?idxno=...   
1   http://www.m-i.kr/news/articleView.html?idxno=...   
2   https://biz.newdaily.co.kr/site/data/html/2024...   
3   https://www.newsis.com/view/?id=NISX20240426_0...   
4   http://weekly.chosun.com/news/articleView.html...   
5   http://www.kyeongin.com/main/view.php?key=2024...   
6           https://www.mediapen.com/news/view/912145   
7   https://www.newsquest.co.kr/news/articleView.h...   
8   https://biz.chosun.com/topics/topics_social/20...   
9   http://tbs.seoul.kr/news/newsView.do?typ_800=7...   
10  https://www.khan.co.kr/national/national-gener...   
11  https://www.donga.com/news/Culture/article/all...   
12  https://www.newsis.com/view/?id=NISX20240319_0...   
13  https://www.dailian.co.kr/news/view/1337742/?s...   
14  http://www.labortoday.co.kr/news/articleView.h...   
15  https://news.kbs.co.kr/news/pc/view/view.do?nc...   
16  https://www.obsnews.co.kr/news/articleView.htm...   
17  https://imnews.imbc.com/replay/2024/nwdesk/art...   
18  http://news.mt.co.kr/mtview.php?no=20240222163...   
19        https://www.sedaily.com/NewsView/2D5GKPHTS3   
20              https://www.mk.co.kr/article/11013698   
21              https://www.mk.co.kr/article/11013565   
22  http://www.dailyimpact.co.kr/news/articleView....   
23  https://www.dt.co.kr/contents.html?article_no=...   
24  https://www.obsnews.co.kr/news/articleView.htm...   
25  https://www.korea.kr/news/reporterView.do?news...   
26  http://www.sisajournal.com/news/articleView.ht...   
27  http://www.hansbiz.co.kr/news/articleView.html...   
28  https://www.asiatime.co.kr/article/20240502500403   
29  https://www.incheonilbo.com/news/articleView.h...   

                                                title   authors  \
0                      [기획] “먹거리에 공공요금까지”… 물가 도미노 현실화        []   
1          [기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑        []   
2               김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격        []   
3                 가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②]        []   
4                                   사모펀드에 포획된 버스 준공영제        []   
5   김교흥 '제2 경의선숲길', 이용우 '서울 통근 교통혁명', 모경종 '5호선 연장 ...        []   
6                       1호선 의왕~당정역 구간서 사망사고…상행선 운행 차질        []   
7             '총선이 끝나고 난 뒤~' 에너지요금 '꿈틀'...대중교통·의료대란은?        []   
8               ‘파업하면 올스톱’ 법 구멍에 서울 버스기사 월급 500만원 넘었다        []   
9                 서울 시내버스 파업에 출퇴근대란 없도록…'필수공익사업' 지정될까  [최가영 기자]   
10          장애인 시위 ‘지하철 탑승→승강장 눕기’로 바꿨지만···경찰 대응은 그대로        []   
11                                 경의중앙선을 위한 변명[소소칼럼]        []   
12                   또 멈춘 수도권지하철…"운행차량 칸 절반이상 20년 넘어"        []   
13      서울지하철 정시도착률 살펴보니…5·6·7호선 '100점' [데일리안이 간다 36]        []   
14     [김영삼 정권시대의 노동운동 ②] 유례 없는 총파업, 막지 못한 신자유주의 노동체제        []   
15                                             KBS 뉴스        []   
16                             폭설에 서울지하철 운행 차질…사고 잇따라        []   
17             서울에 내린 10cm 넘는 폭설‥출근길 혼잡·지하철 지연운행도 잇따라        []   
18                 폭설로 지하철 지연…"일찍 나왔는데 면접 지각" 누리꾼 하소연        []   
19            [기습폭설에 사고 속출] '습설'에 건물 붕괴…미끄러진 차에 목숨 잃어        []   
20                           GTX·5호선 연장 호재 … 김포 집값 쑥쑥        []   
21                GTX·5호선 연장 호재…‘이 지역’ 집값 슬금슬금 계속 오르네        []   
22           예상 뛰어넘는 K-패스 인기에...카드업계 '미소' < 금융 < 기사본문        []   
23                  귀해지는 수도권 아파트 물량 속 승승장구하는 정비사업 아파트        []   
24                        5호선 연장안 이달 확정되나? 대광위 중재력 관건        []   
25                                 대중교통, 난 K-패스로 통한다!        []   
26                   “분양가 내릴 기미 안 보여”…무순위·입주권 찾는 수요자들        []   
27  “하남시-강동구 9호선 연장하라!”...경기도에 일반열차 연장 운행 및 조속개통 공...        []   
28            [증권소식] 작년 증권사 해외법인 순익 1814억원…전년비 11% 증가        []   
29                               9호선 일반열차 연장·조속 개통 촉구        []   

                 publish_date  \
0   2024-05-02 12:00:00+09:00   
1   2024-05-02 14:39:44+09:00   
2         2024-04-28 00:00:00   
3   2024-04-29 15:09:44+09:00   
4   2024-04-26 15:00:00+09:00   
5     

# 4. 데이터 전처리
- publish_date의 null 값 처리를 위해 한번더 selenium 진행

In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from dateutil import parser
import re

# Setup Chrome WebDriver with headless options and other configurations
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('blink-settings=imagesEnabled=false')  # 이미지 비활성화

# Setup service with ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(120)  # 타임아웃을 60초로 설정

# Assume 'df' is your DataFrame and it has columns 'url' and 'publish_date'
date_classes = ['news-date', 'date-repoter', 'article_info', 'article_byline', 'dates']

for index, row in df.iterrows():
    if pd.isna(row['publish_date']):
        url = row['url']
        try:
            driver.get(url)
        except TimeoutException:
            print(f"Timeout while loading {url}")
            continue  # 다음 URL로 건너뛰기
        
        publish_date = None
        for date_class in date_classes:
            date_elements = driver.find_elements(By.CLASS_NAME, date_class)
            for date_element in date_elements:
                date_text = date_element.text
                numbers = re.findall(r'\d+', date_text)
                if len(numbers) >= 3:  # YYYY, MM, DD가 필요
                    publish_date = f"{numbers[0]}-{numbers[1].zfill(2)}-{numbers[2].zfill(2)}"
                    break
            if publish_date:
                break
        
        if publish_date:
            try:
                parsed_date = parser.parse(publish_date)
                df.at[index, 'publish_date'] = parsed_date
            except ValueError as e:
                print(f"Error parsing date from {url}: {e}")

# Close the browser
driver.quit()


In [36]:
df.head(30)

url  \
0   http://www.m-i.kr/news/articleView.html?idxno=...   
1   http://www.m-i.kr/news/articleView.html?idxno=...   
2   https://biz.newdaily.co.kr/site/data/html/2024...   
3   https://www.newsis.com/view/?id=NISX20240426_0...   
4   http://weekly.chosun.com/news/articleView.html...   
5   http://www.kyeongin.com/main/view.php?key=2024...   
6           https://www.mediapen.com/news/view/912145   
7   https://www.newsquest.co.kr/news/articleView.h...   
8   https://biz.chosun.com/topics/topics_social/20...   
9   http://tbs.seoul.kr/news/newsView.do?typ_800=7...   
10  https://www.khan.co.kr/national/national-gener...   
11  https://www.donga.com/news/Culture/article/all...   
12  https://www.newsis.com/view/?id=NISX20240319_0...   
13  https://www.dailian.co.kr/news/view/1337742/?s...   
14  http://www.labortoday.co.kr/news/articleView.h...   
15  https://news.kbs.co.kr/news/pc/view/view.do?nc...   
16  https://www.obsnews.co.kr/news/articleView.htm...   
17  https://imnews.imbc.com/replay/2024/nwdesk/art...   
18  http://news.mt.co.kr/mtview.php?no=20240222163...   
19        https://www.sedaily.com/NewsView/2D5GKPHTS3   
20              https://www.mk.co.kr/article/11013698   
21              https://www.mk.co.kr/article/11013565   
22  http://www.dailyimpact.co.kr/news/articleView....   
23  https://www.dt.co.kr/contents.html?article_no=...   
24  https://www.obsnews.co.kr/news/articleView.htm...   
25  https://www.korea.kr/news/reporterView.do?news...   
26  http://www.sisajournal.com/news/articleView.ht...   
27  http://www.hansbiz.co.kr/news/articleView.html...   
28  https://www.asiatime.co.kr/article/20240502500403   
29  https://www.incheonilbo.com/news/articleView.h...   

                                                title   authors  \
0                      [기획] “먹거리에 공공요금까지”… 물가 도미노 현실화        []   
1          [기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑        []   
2               김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격        []   
3                 가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②]        []   
4                                   사모펀드에 포획된 버스 준공영제        []   
5   김교흥 '제2 경의선숲길', 이용우 '서울 통근 교통혁명', 모경종 '5호선 연장 ...        []   
6                       1호선 의왕~당정역 구간서 사망사고…상행선 운행 차질        []   
7             '총선이 끝나고 난 뒤~' 에너지요금 '꿈틀'...대중교통·의료대란은?        []   
8               ‘파업하면 올스톱’ 법 구멍에 서울 버스기사 월급 500만원 넘었다        []   
9                 서울 시내버스 파업에 출퇴근대란 없도록…'필수공익사업' 지정될까  [최가영 기자]   
10          장애인 시위 ‘지하철 탑승→승강장 눕기’로 바꿨지만···경찰 대응은 그대로        []   
11                                 경의중앙선을 위한 변명[소소칼럼]        []   
12                   또 멈춘 수도권지하철…"운행차량 칸 절반이상 20년 넘어"        []   
13      서울지하철 정시도착률 살펴보니…5·6·7호선 '100점' [데일리안이 간다 36]        []   
14     [김영삼 정권시대의 노동운동 ②] 유례 없는 총파업, 막지 못한 신자유주의 노동체제        []   
15                                             KBS 뉴스        []   
16                             폭설에 서울지하철 운행 차질…사고 잇따라        []   
17             서울에 내린 10cm 넘는 폭설‥출근길 혼잡·지하철 지연운행도 잇따라        []   
18                 폭설로 지하철 지연…"일찍 나왔는데 면접 지각" 누리꾼 하소연        []   
19            [기습폭설에 사고 속출] '습설'에 건물 붕괴…미끄러진 차에 목숨 잃어        []   
20                           GTX·5호선 연장 호재 … 김포 집값 쑥쑥        []   
21                GTX·5호선 연장 호재…‘이 지역’ 집값 슬금슬금 계속 오르네        []   
22           예상 뛰어넘는 K-패스 인기에...카드업계 '미소' < 금융 < 기사본문        []   
23                  귀해지는 수도권 아파트 물량 속 승승장구하는 정비사업 아파트        []   
24                        5호선 연장안 이달 확정되나? 대광위 중재력 관건        []   
25                                 대중교통, 난 K-패스로 통한다!        []   
26                   “분양가 내릴 기미 안 보여”…무순위·입주권 찾는 수요자들        []   
27  “하남시-강동구 9호선 연장하라!”...경기도에 일반열차 연장 운행 및 조속개통 공...        []   
28            [증권소식] 작년 증권사 해외법인 순익 1814억원…전년비 11% 증가        []   
29                               9호선 일반열차 연장·조속 개통 촉구        []   

                 publish_date  \
0   2024-05-02 12:00:00+09:00   
1   2024-05-02 14:39:44+09:00   
2         2024-04-28 00:00:00   
3   2024-04-29 15:09:44+09:00   
4   2024-04-26 15:00:00+09:00   
5     

In [37]:
df.to_csv('knowledge_generation_data.csv', index=False)